In [15]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")
load("vertex_2d.sage")

rank=3




KT_vars = ['x','y']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)

x,y=KT_ring.gens()
t,m = coho_ring.gens()
b=var('b')
w=SR.var('w',rank)
v=SR.var('v',rank)


def character(p,x=x,y=y):
    return sum(y**(-i) * x**(-j) for i, j in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))


def tangent(p,x=x,y=y):
    total=0
    for b1,b2 in p.cells():
        arm=p[b1]-b2-1
        leg=p.conjugate()[b2]-b1-1
        total+=y^(-leg)*x^(arm+1)+y^(leg+1)*x^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)

def canonical_bundle_red(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)-1

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def total_chern(f):
    numer, denom = from_character(f)
    new_numer=[i+1 for i in numer]
    new_denom=[i+1 for i in denom]
    return coho_ring.fraction_field()(prod(new_numer+[1]) / (prod(new_denom)))

def chern_class(k, f):
    roots = sum((c*[from_monomial(w)] for c, w in f), [])
    return sum(prod(S) for S in itertools.combinations(roots, k))

def Nek(f, check=False):
    R = f.parent()
    numer, denom = R.one(), R.one()
    for exp, coeff in f.dict().items():
        if exp.is_constant():
            term = R.zero()
        else:
            term = R({exp: 1, exp.emul(0): -1})
        if coeff > 0:
            numer *= term ** (coeff)
        elif coeff < 0:
            denom *= term ** (-coeff)
    return R.fraction_field()(numer) / denom

def dual(f, check=False):
    R = f.parent()
    g=R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def highest_degree_y(f):
    R = f.parent()
    max_deg=0
    for exp, coeff in f.dict().items():
        if exp[1]>max_deg:
            max_deg = exp[1]
    term=R.zero()
    for exp, coeff in f.dict().items():
        if exp[1]==max_deg:
            term+=R({exp:coeff})
    return term

def lowest_degree_y(f):
    R = f.parent()
    min_deg=0
    for exp, coeff in f.dict().items():
        if exp[4]<min_deg:
            min_deg = exp[4]
    term=R.zero()
    for exp, coeff in f.dict().items():
        if exp[4]==min_deg:
            term+=R({exp:coeff})
    return term
    

def top_chern_vb(p):
    Q = character(p,x,y=1/x)
    roots = []
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        
        for i in range(rank):
            roots += (coeff) * [wt+w[i]*t]
    total = prod((1+r) for r in roots)
    return total

def total_segre_vb(p):
    Q = character(p,x,y=1/x)
    roots = []
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        for i in range(rank):
            roots += (coeff) * [wt+w[i]*t]
    total = prod(1/(1+r) for r in roots)
    return total
    
def determinant_vb(p,n):
    Q = character(p,x,y=1/x)
    det = (determinant(Q)^rank)*((SR(x)^(sum(w[i] for i in range(rank))))^n)
    return det(x=exp(t*b))
    
def Chern_Num(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent(p,x,1/x))*top_chern_vb(p)*top_chern(dual(canonical_bundle_red(p,x,1/x))))(t=t*b)
        total+=add
    return total


def Segre_Num(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent(p,x,1/x))*total_segre_vb(p)*top_chern(dual(canonical_bundle_red(p,x,1/x))))(t=t*b)
        total+=add
    return total

def Verlinde_Num(n):
    total=0
    for p in Partitions(n):
        add=(KTheory.measure_unsymmetrized(tangent(p,x,1/x))(x=exp(t*b))*\
            determinant_vb(p,n)*\
            KTheory.measure_unsymmetrized(-canonical_bundle_red(p,x,1/x),inv=False)(x=exp(t*b))).taylor(b,0,4)
        total+=add
    return total

def DT_Num(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent(p,x,1/x))*top_chern(dual(canonical_bundle_red(p,x,1/x))))(t=t*b)
        total+=add
    return total

def Verlinde_Num_inv(n):
    total=0
    for p in Partitions(n):
        add=(KTheory.measure_unsymmetrized(tangent(p,x,1/x))(x=exp(t*b))*\
            (1/determinant_vb(p,n))*\
            KTheory.measure_unsymmetrized(-canonical_bundle_red(p,x,1/x),inv=False)(x=exp(t*b))).taylor(b,0,4)
        total+=add
    return total
        

def degn(f,n):
    g=f.taylor(b,0,n)
    L=g.coefficients(b)
    if L[len(L)-1][1]!=n:
        return 0
    return L[len(L)-1][0]



def total_segre_vb_2d(p):
    Q = character(p,x,y)
    roots = []
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        for i in range(rank):
            roots += (coeff) * [wt+w[i]*t+v[i]*m]
    total = prod(1/(1+r) for r in roots)
    return total

def Segre_Num_2d(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent(p))*total_segre_vb_2d(p)*top_chern(dual(canonical_bundle(p,x,y))))(t=t*b,m=m*b)
        total+=add
    return total

def determinant_vb_2d(p,n):
    Q = character(p,x,y)
    det = (determinant(Q)^rank)*((SR(x)^(sum(w[i] for i in range(rank)))*(SR(y)^(sum(v[i] for i in range(rank)))))^n)
    return det(x=exp(t*b),y=exp(m*b))

def Verlinde_Num_2d(n):
    total=0
    for p in Partitions(n):
        add=(KTheory.measure_unsymmetrized(tangent(p))(x=exp(t*b),y=exp(m*b))*\
            determinant_vb_2d(p,n)*\
            KTheory.measure_unsymmetrized(-canonical_bundle(p,x,y),inv=False)(x=exp(t*b),y=exp(m*b))).taylor(b,0,4)
        total+=add
    return total

def DT_Num_2d(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent(p))*top_chern(dual(canonical_bundle(p,x,y))))(t=t*b,m=m*b)
        total+=add
    return total

def top_chern_vb_2d(p):
    Q = character(p,x,y)
    roots = []
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        for i in range(rank):
            roots += (coeff) * [wt+w[i]*t+v[i]*m]
    total = prod(r for r in roots)
    return total

def Chern_Num_2d(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent(p))*top_chern_vb_2d(p)*top_chern(dual(canonical_bundle(p,x,y))))
        total+=add
    return total

def character_vb_rank_2(p,x=x,y=y):
    Q = character(p,x,y)
    return Q * (exp(w[0])+exp(v[0]))

def measure_z_vb(p):
    f = character_vb_rank_2(p,x,y)
    L = f.expand().operands()
    total=1
    for monomial in L:
        term = 1 - monomial*z
        total*=term
    return total

def Master_Partition_rank_2(n):
    total=0
    for p in Partitions(n):
        add=determinant(dual(character_vb(p,(0,0),x,y)))*\
            measure_unsymmetrized(tangent(p))*\
            measure_z_vb(p)
        total+=add
    return (-1)^n*total

In [549]:
pretty(pretty(Master_Partition_rank_2(1))(z=1/(1+z))(x=exp(-t*z),y=exp(-m*z),v0=-v[0]*z,w0=-w[0]*z).taylor(z,0,1))(m=-t)

-1/2*(v0^2*w0*z + v0*w0^2*z + v0^2*z + 4*v0*w0*z + w0^2*z - 2*v0*w0 + 4*v0*z + 4*w0*z - 2*v0 - 2*w0 + 4*z - 2)/t^2

In [543]:
degn(Segre_Num(1),0)

-w0^2 - w0*w1 - w1^2

In [3]:
degn(Chern_Num(1),-1)

-(w0 + w1)/t

We try to get the Verlinde-Segre correspondence for $\mathbb{C}^2$.

Over $\mathbb{C}^2$, the equivariant cohomology ring is $\mathbb{Z}[t_1,t_2]/(t_1+t_2)=\mathbb{Z}[t]$. The equivariant K-theory ring is $\mathbb{Z}[x^{\pm1}]$. The Zariski tangent space at $Z_\pi$ is
$$Q+\bar Q-Q\bar Q(1-x)(1-1/x) $$
where $Q$ is the character of partition $\pi$.

The virtual Segre number is 
$$\int_{[S^{[n]}]^{\text{vir}}} s(\alpha^{[n]})=\sum_{|\pi|=n}e(-T|_Z)s(\alpha^{[n]}|_Z)c_{\text{top}}(K_S^{[n]}|_Z) $$

The Verlinde number is 
$$V_n(\alpha)= \chi(\det (\alpha^{[n]}))=\int_{[S^{[n]}]^{\text{vir}}}\text{td}(T^{\text{vir}})\text{ch}\left( \det (\alpha^{[n]})\right)$$
$$=\sum_{|\pi|=n}e(-T^{\text{vir}}|_Z)\text{td}(T^{\text{vir}}|_Z)\text{ch}\left( \det (\alpha^{[n]})|_Z\right)$$
$$=\sum_{|\pi|=n}\frac{\text{ch}(\wedge K^{[n]}_S|_Z)}{\text{ch}(\wedge T^\vee|_Z)}\text{ch}\left( \det (\alpha^{[n]})|_Z\right) $$
where ${\text{ch}(\wedge x^\vee)}=1/(1-e^{-x})$

Since on $\mathbb{C}^2$, $K_S$ is trivial with no weight from the 1-d torus action, we know $K^{[n]}$ has non-trivial fixed parts, so the above invariaints are 0 (in degree 0 of $H^*_{\mathbb{C}^*}(\text{pt})$). Instead we take the "reduced" obstruction bundle:
$$ K^{[n]}_S-\mathcal{O}_{S^{[n]}}$$
Now we have in degree 0 they agree for $n=1,2,3$. When $n>3$, the fixed part of $K^{[n]}$ has dimension at least 2 at some boxes, but the equality continue to hold.

The degree $-1$ term for Segre is $-(w\ /\ nt)$:

In [524]:
pretty(degn(Chern_Num(1),0))

-w0*w1

In [526]:
pretty(degn(Chern_Num_2d(1),0))

(m*v0 + t*w0)*(m*v1 + t*w1)*(m + t)/(m*t)

In [503]:
pretty(degn(Verlinde_Num(6),0))

-231*w0^2 - 154

In [447]:
pretty(degn(Segre_Num(3),0))

-24*w0^2 - 36*w0*w1 - 24*w1^2 - 22

In [5]:
pretty(degn(Verlinde_Num_inv(3),0))

-5*w0^2 - 10*w0*w1 - 5*w1^2 - 5/6

The Verlinde number is $(-1)^{n+1}(w\ /\ nt)$:

In [153]:
pretty(degn(Verlinde_Num_inv(2),-1))

-5/2*(w0 + w1 + w2)/t

and we indeed have correspondence. Following is segre when $rank>1$:

In [152]:
pretty(degn(Segre_Num(3),-1))

55/3*(w0 + w1 + w2)/t

In [157]:
pretty(degn(Verlinde_Num(3),-1))

-55/3*(w0 + w1 + w2)/t

Back to rank $1$, then the segre number $1/(1+x)$ is an infinite power series that has terms in other degrees.

In [101]:
pretty(degn(Segre_Num(2),-2))

1/2/t^2

In [353]:
pretty(degn(Verlinde_Num(2),-2))

-5/4/t^2

In degree $-2$, the difference (quotient) is
$$\frac{V_n}{S_n}=(-1)^{n+1} \frac{(2n-1)}{n}$$

Below degree $-2$, everything vanishes, which might be non-trivial, perhaps due to $K^{[n]}$ containing trivial summand for partitions that contain the cell $(1,1)$, which makes the partitions that count very symmetrical.

In degree $0$, the invariants are always polynomials in $w^2$:
$$[w^2]S_n=(-1)^{n+1}\cdot2\cdot [w^2]V_{n} $$
$$\frac{[w^2]S_n}{[w^0]S_n} = \frac{[w^2]V_{n+1}}{[w^0]V_{n+1}}, \text{ for $n\geq 2$ (otherwise division by 0)} $$

In [13]:
pretty(degn(Verlinde_Num(3),0))

-5*w0^2 - 5/6

In [14]:
pretty(degn(Segre_Num(3),0))

-10*w0^2 - 10/3

In [384]:
pretty(degn(Segre_Num(1),0))

-w0^2 - w0*w1 - w1^2 - w0*w2 - w1*w2 - w2^2

In [385]:
pretty(degn(Segre_Num(2),0))

6*w0^2 + 8*w0*w1 + 6*w1^2 + 8*w0*w2 + 8*w1*w2 + 6*w2^2 + 5

In [386]:
pretty(degn(Segre_Num(3),0))

-44*w0^2 - 66*w0*w1 - 44*w1^2 - 66*w0*w2 - 66*w1*w2 - 44*w2^2 - 77

In [387]:
pretty(degn(Segre_Num(4),0))

350*w0^2 + 560*w0*w1 + 350*w1^2 + 560*w0*w2 + 560*w1*w2 + 350*w2^2 + 945

In [388]:
pretty(degn(Segre_Num(5),0))

-2907*w0^2 - 4845*w0*w1 - 2907*w1^2 - 4845*w0*w2 - 4845*w1*w2 - 2907*w2^2 - 10659

In [389]:
pretty(degn(Segre_Num(6),0))

24794*w0^2 + 42504*w0*w1 + 24794*w1^2 + 42504*w0*w2 + 42504*w1*w2 + 24794*w2^2 + 115115

In [358]:
pretty(degn(Verlinde_Num_inv(2),0))

5/2*w0^2 + 5*w0*w1 + 5/2*w1^2 + 5*w0*w2 + 5*w1*w2 + 5/2*w2^2 + 5/6

In [359]:
pretty(degn(Verlinde_Num(2),0))

-7/2*w0^2 - 7*w0*w1 - 7/2*w1^2 - 7*w0*w2 - 7*w1*w2 - 7/2*w2^2 - 35/12

In [360]:
pretty(degn(Verlinde_Num(3),0))

-55/2*w0^2 - 55*w0*w1 - 55/2*w1^2 - 55*w0*w2 - 55*w1*w2 - 55/2*w2^2 - 605/12

In [361]:
pretty(degn(Verlinde_Num(4),0))

-455/2*w0^2 - 455*w0*w1 - 455/2*w1^2 - 455*w0*w2 - 455*w1*w2 - 455/2*w2^2 - 7735/12

In [362]:
pretty(degn(Verlinde_Num(5),0))

-1938*w0^2 - 3876*w0*w1 - 1938*w1^2 - 3876*w0*w2 - 3876*w1*w2 - 1938*w2^2 - 7429

In [205]:
pretty(degn(Verlinde_Num(6),0))

-231*w0^2 - 154

In [214]:
pretty(degn(Verlinde_Num_inv(2),-1))

-1/2*w0/t

Also, we have a recursive formula
$$\frac{[w^2]S_n}{[w^0]S_n}\cdot \frac{(n-1)}{n} = \frac{[w^2]S_{n+1}}{[w^0]S_{n+1}}, n\geq 2$$

In degree $1$ rank $1$, 
$$S_n=(-1)^nV_{n+1}$$

In [105]:
pretty(degn(Segre_Num(3),1))

70/3*(w0^2 + 1)*t*w0

In [17]:
pretty(degn(Verlinde_Num(4),0))

-455/2*w0^2 - 455*w0*w1 - 455/2*w1^2 - 455*w0*w2 - 455*w1*w2 - 455/2*w2^2 - 7735/12

But not for other ranks.

# DT inv

We have
$$\int_{[S^{[n]}]}1=(-1)^n\frac{1}{n!t^{2n}}$$
$$\int_{[S^{[n]}]^{\text{red}}}1=\begin{cases}-\frac1{t^2}\text{ when $n=1$}\\0\text{ else}\end{cases}$$

In [119]:
DT_Num(1)

-1/(b^2*t^2)

In [120]:
pretty(DT_Num_2d(4))

1/24*(m + t)^4/(b^4*m^4*t^4)

# 2d torus
Compare with universal series from [Khilb] 

In [128]:
pretty(degn(Segre_Num_2d(2),0))

1/2*(3*m^3*v0^2 + 3*m^2*t*v0^2 + 6*m^2*t*v0*w0 + 6*m*t^2*v0*w0 + 3*m*t^2*w0^2 + 3*t^3*w0^2 + 3*m^2*t*v0 + 3*m*t^2*w0 - 2*m^2*t - 2*m*t^2)*(m + t)/(m^2*t^2)

In [79]:
def var_change(k):
    return degn(pretty(sum(degn(Chern_Num_2d(i),0)*b^i for i in range(1,k+1)))(b=b),k)

In [130]:
pretty(degn(Segre_Num_2d(3),0))(v0=0,w0=0)

-1/2*(2*m^4*t - 3*m^3*t^2 - 3*m^2*t^3 + 2*m*t^4)*(m + t)/(m^3*t^3)

NameError: name 'Vertex2d' is not defined